In [1]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel  # as parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as visionUtils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
# ---------------------------------------- #
from IPython.display import HTML


In [2]:
# set random seed = 999
manualSeed = 999  # we can set randomly or manually  {manualSeed = random.randint(1, 10000)
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

Random Seed:  999


### Set Hyper-Parameters

In [3]:
dataroot = './data/celeba'
os.makedirs(dataroot, exist_ok=True)
workers = 2     # num of workers for dataloaders ???

batch_size = 128
image_size = 64  # depend on how large the image is
in_channels = 3  # depend on how mang channels of the image (r, g, b)
latent = 100    # size of latent vector (z)
gen_fea = 64    # size of feature map in generator
dis_fea = 64    # **** in discriminator
num_epochs = 5  # training epochs
lr = 0.002     # optimizer learning rate for "Adam"
beta1 = 0.5     # beta1 for adam
n_gpu = 0       # no GPU using

In [4]:
# download the dataset cele_a or load the dataset
cele_a = dset.ImageFolder(root=dataroot,
                     transform=(
                         transforms.Compose([
                             transforms.Resize(image_size),
                             transforms.CenterCrop(image_size),
                             transforms.ToTensor(),
                             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5), inplace=False)
                             # here we don't use the "inplace = True" is because the transforms is assigned to cele_a, which create a new one instead of making changes on the original dataset
                         ])
                     ),
                          ) # dset.CelebA(..., download=True)

# Create the dataloader
dataloader = data.DataLoader(cele_a, batch_size=batch_size, shuffle=True, sampler=None, num_workers=workers)    # pin_memory = False  because there 's no GPU on this computer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # cuda or cpu


print(dataloader.dataset, len(cele_a), len(dataloader))

Dataset ImageFolder
    Number of datapoints: 202599
    Root location: ./data/celeba
    StandardTransform
Transform: Compose(
               Resize(size=64, interpolation=bilinear, max_size=None, antialias=None)
               CenterCrop(size=(64, 64))
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           ) 202599 1583


In [5]:
# print(len(cele_a), len(dataloader))
# print(len(cele_a.classes))  # only 1 class in this dataset
# print(type(cele_a[0][0]))
# print(dataloader.__dict__)
# print(dataloader.__doc__)
# print(type(dataloader),
#       type(dataloader.dataset[0]),
#       type(dataloader.dataset[0][0]))

# img0 = dataloader.dataset[0][0]
# print(img0.device)
# a = next(iter(dataloader))


# plt.figure()    # figsize=(8, 8)
# plt.imshow(np.transpose(visionUtils.make_grid(img0, padding=0, normalize=True).cpu(), (1,2,0)))

In [6]:
# # print(a[0][:64])
# plt.figure(figsize=(8,8))
# plt.axis('off')
#
# # normalize back to the image
# # 1.
# img = a[0][0] * 128 + 128
# plt.imshow(np.transpose(img / 255, axes=(1, 2, 0)))

# 2.
# plt.imshow(np.transpose(visionUtils.make_grid(a[0][:64], padding=0, normalize=True), axes=(1, 2, 0)))

##### Original Code for normalize the images to imshows

In [7]:
# Plot some training images
# real_batch = next(iter(dataloader))
# plt.figure(figsize=(8,8))
# plt.axis("off")
# plt.title("Training Images")
# plt.imshow(np.transpose(visionUtils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

# print(type(real_batch[0]), len(real_batch[0]))

### Weight Initialization

In [8]:
# custom weight initialization on netG and netD
# what for ??? why this format ???

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

### Generator
<img src="../img_indoc/dcgan_generator.png" alt="generator model" width="700">

In [9]:
# ![generator model](../img_indoc/dcgan_generator.png)

In [10]:
from model import Generator, Discriminator
net_G = Generator(latent, gen_fea, in_channels, num_gpu=n_gpu).to(device)  # generate in_channels size imgs
net_D = Discriminator(gen_fea=gen_fea, channels=in_channels, num_gpu=n_gpu).to(device) # distinguish images in in_channels size

# multi-gpu if desired (Data Parallel, not model parallel or )
if (device.type == "cuda") and (n_gpu > 1):
    net_G = nn.DataParallel(net_G, list(range(n_gpu)))
    net_D = nn.DataParallel(net_D, list(range(n_gpu)))

net_G.apply(weights_init)
net_D.apply(weights_init)

# print(net_G, "\n", net_D)

Discriminator(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
    (6): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (9): Sigmoid()
  )
)

### Loss function and Optimizer

In [11]:
criterion = nn.BCELoss() # binary cross-entropy loss function
# loss = -y * log(D(x)) - (1 - y) * log(1 - D(G(z))) where y is label, D(x) is prediction of real images and D(G(z)) is prediction of fake images
# loss of generator => maximize the log(D(G(z)) to deceive the Discriminator the output is the real image
#       close to 0.5 is best, because that means the discriminator(which is also learn in this process) cannot tell which image is real or fake

fixed_noise = torch.randn(64, latent, 1, 1, device=device)
real_label = 1; fake_label = 0;

# Setup Optimizer
optim_G = optim.Adam(net_G.parameters(), lr=lr, betas=(beta1, 0.999))
optim_D = optim.Adam(net_D.parameters(), lr=lr, betas=(beta1, 0.999))
# optim_G = optim.SGD(net_G.parameters(), lr=lr, momentum=0.1)
# optim_D = optim.SGD(net_D.parameters(), lr=lr, momentum=0.1)

### Train

In [12]:
img_list = []
G_losses = []
D_losses = []
iters = 0

print("Starting Training Looping...")
# For each epoch
for epoch in range(num_epochs):
    # For each batch in the dataloader
    for i, data in enumerate(dataloader, start=0):
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        net_D.zero_grad()

        #format batch
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size, ), real_label, dtype=torch.float, device=device)
        # forward pass real batch through D
        output = net_D(real_cpu).view(-1)
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        noise = torch.randn(b_size, latent, 1, 1, dtype=torch.float, device=device)
        fake = net_G(noise)     # prediction for fake images
        # print(noise.shape, fake.shape)
        label.fill_(fake_label) # change label data from 1 -> 0
        output = net_D(fake.detach()).view(-1)
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = (errD_fake + errD_real) / 2
        # print(errD_real.detach(), errD_fake.detach())

        optim_D.step()


        # (2) Update G network: maximize log(D(G(z)))
        net_G.zero_grad()
        label.fill_(real_label)
        output = net_D(fake).view(-1)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()

        optim_G.step()

        if i % 50 == 0:
            print('[%d/%d] [%d/%d]\t loss_d: %.4f\t loss_g: %.4f\t D(x): %.4f\t D(G(z)): %.4f/%.4f' % (epoch, num_epochs, i, len(dataloader), errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
            G_losses.append(errG.item())
            D_losses.append(errD.item())

        if (iters % 500 == 0) or ((epoch == num_epochs - 1) and (i == len(dataloader) - 1)):
            with torch.no_grad():
                fake = net_G(fixed_noise).detach().cpu()
            img_list.append(visionUtils.make_grid(fake, padding=2, normalize=True))

        iters += 1

Starting Training Looping...


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Users/bling/opt/anaconda3/envs/cv/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Users/bling/opt/anaconda3/envs/cv/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
  File "/Users/bling/opt/anaconda3/envs/cv/lib/python3.9/site-packages/torch/__init__.py", line 711, in <module>
    from torch import hub as hub
  File "/Users/bling/opt/anaconda3/envs/cv/lib/python3.9/site-packages/torch/hub.py", line 18, in <module>
    from tqdm.auto import tqdm  # automatically select proper tqdm submodule if available
  File "/Users/bling/opt/anaconda3/envs/cv/lib/python3.9/site-packages/tqdm/__init__.py", line 3, in <module>
    from .cli import main  # TODO: remove in v5.0.0
  File "/Users/bling/opt/anaconda3/envs/cv/lib/python3.9/site-packages/tqdm/cli.py", line 9, in <module>
    from .std imp

KeyboardInterrupt: 

In [ ]:
# Grab a batch of real images from the dataloader
real_batch = next(iter(dataloader))

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(visionUtils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()